In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers
import datetime
import timeit
import itertools
import pyarrow as pa

2025-10-10 19:33:24.830673: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-10 19:33:24.868884: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-10 19:33:25.863332: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
print("TensorFlow:", tf.__version__)
print("NumPy:", np.__version__)
print("pandas:", pd.__version__)
print("pyarrow:", pa.__version__)

TensorFlow: 2.20.0
NumPy: 2.3.3
pandas: 2.3.3
pyarrow: 21.0.0


In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

/home/has/sources/components/setmy.info/submodules/setmy-info.github.io/src/site/markdown/ai/.venv/lib/python3.13/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-10-10 19:33:35.374693: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.3979766 , -0.3679855 ,  0.1885733 , -0.5292295 ,  0.16136673,
         0.29011023,  0.6836933 , -0.47953945, -1.1442078 , -0.42209604]],
      dtype=float32)

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.07264147, 0.07485307, 0.13059303, 0.06370628, 0.12708795,
        0.14454962, 0.21426336, 0.06695181, 0.03444297, 0.07091036]],
      dtype=float32)

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [8]:
loss_fn(y_train[:1], predictions).numpy()

np.float32(1.9341323)

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [10]:
model.fit(x_train, y_train, epochs=7)

Epoch 1/7
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9148 - loss: 0.2946
Epoch 2/7
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9566 - loss: 0.1424
Epoch 3/7
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9668 - loss: 0.1088
Epoch 4/7
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9728 - loss: 0.0888
Epoch 5/7
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9764 - loss: 0.0753
Epoch 6/7
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9786 - loss: 0.0662
Epoch 7/7
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9812 - loss: 0.0589


In [11]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - 1ms/step - accuracy: 0.9806 - loss: 0.0640


[0.06400463730096817, 0.9805999994277954]

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[4.57731737e-08, 2.95014555e-08, 2.66790579e-07, 2.20684058e-04,
        2.66578816e-12, 3.27941876e-07, 4.40294258e-13, 9.99754846e-01,
        1.17136472e-06, 2.25995755e-05],
       [2.41008919e-08, 1.28064210e-06, 9.99998689e-01, 4.31410996e-09,
        2.38294962e-21, 1.99164596e-08, 3.16556070e-09, 1.15824624e-14,
        3.75937041e-08, 1.29909994e-15],
       [5.29833954e-08, 9.99585330e-01, 2.19914637e-05, 9.80889240e-07,
        2.07913649e-06, 1.34711581e-06, 4.62798255e-07, 3.72390234e-04,
        1.54973677e-05, 1.10516165e-08],
       [9.99938130e-01, 5.57821541e-12, 3.48784233e-05, 3.64611208e-09,
        8.24895707e-09, 1.38410428e-06, 1.18769640e-05, 1.06429854e-06,
        1.54233728e-08, 1.25797242e-05],
       [5.29858662e-05, 6.42628339e-10, 2.48487936e-06, 1.53745763e-08,
        9.79469657e-01, 2.42738398e-08, 7.55401061e-06, 2.07065401e-04,
        1.13407132e-05, 2.02488862e-02]], dtype=float32)>